In [1]:
#load python packages
import os
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [2]:
import re

In [3]:
os.getcwd()

'C:\\Users\\craig\\Documents\\Python Scripts\\Notebooks\\Loan_Data_Classification\\Data_Wrangaling'

In [4]:
path="C:\\Users\\craig\\Documents\\Python Scripts\\Notebooks\\Loan_Data_Classification\\Data"
os.chdir(path) 

In [6]:
#n = 100  # every 100th line = 1% of the lines
#df = pd.read_csv('accepted_2007_to_2018Q4.csv.gz', header=0, skiprows=lambda i: i % n != 0)
#df = pd.read_csv('appl_accepted_20072019Q3.csv')

df = pd.read_csv(
    'accepted_2007_to_2018Q4.csv.gz',
    parse_dates=['issue_d'], infer_datetime_format=True)
df = df[(df.issue_d >= '2016-01-01 00:00:00') & (df.issue_d < '2019-01-01 00:00:00')]
df = df.reset_index(drop=True)
df.head()

MemoryError: Unable to allocate 1.90 GiB for an array with shape (113, 2260701) and data type float64

In [ ]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [ ]:
df.head()

In [ ]:
datadescriptions = pd.read_excel('https://resources.lendingclub.com/LCDataDictionary.xlsx')

In [ ]:
datadescriptions.head(160)

In [ ]:
browsenotes = pd.read_excel('https://resources.lendingclub.com/LCDataDictionary.xlsx', sheet_name=1)

In [ ]:
browsenotes.head(160)

In [ ]:
browse_feature = browsenotes['BrowseNotesFile'].dropna().values
browse_feature = [re.sub('(?<![0-9_])(?=[A-Z0-9])', '_', x).lower().strip() for x in browse_feature]

In [ ]:
df_feature = df.columns.values
np.setdiff1d(browse_feature, df_feature)

In [ ]:
np.setdiff1d(df_feature, browse_feature)

In [ ]:
wrong = ['is_inc_v', 'mths_since_most_recent_inq', 'mths_since_oldest_il_open',
         'mths_since_recent_loan_delinq', 'verified_status_joint']
correct = ['verification_status', 'mths_since_recent_inq', 'mo_sin_old_il_acct',
           'mths_since_recent_bc_dlq', 'verification_status_joint','loan_status']

browse_feature = np.setdiff1d(browse_feature, wrong)
browse_feature = np.append(browse_feature, correct)

In [ ]:
avail_feature = np.intersect1d(browse_feature, df_feature)
df = df[avail_feature].copy()
df.info()

In [ ]:
df.select_dtypes('object').head()

In [ ]:
df['earliest_cr_line'] = pd.to_datetime(df['earliest_cr_line'], infer_datetime_format=True)
df['sec_app_earliest_cr_line'] = pd.to_datetime(df['sec_app_earliest_cr_line'], infer_datetime_format=True)

In [ ]:
df['emp_length'] = df['emp_length'].replace({'< 1 year': '0 years', '10+ years': '11 years'})
df['emp_length'] = df['emp_length'].str.extract('(\d+)').astype('float')
df['id'] = df['id'].astype('float')


In [ ]:
nas=pd.DataFrame(df.isnull().sum().sort_values(ascending=False)/len(df),columns = ['percent'])
pos = nas['percent'] > 0
nas[pos]

In [ ]:
df = df.drop(['desc', 'member_id' ], axis=1, errors='ignore')

In [ ]:
fill_empty = ['emp_title', 'verification_status_joint', 'title', 'zip_code']
fill_max = ['bc_open_to_buy', 'mo_sin_old_il_acct', 'mths_since_last_delinq',
            'mths_since_last_major_derog', 'mths_since_last_record',
            'mths_since_rcnt_il', 'mths_since_recent_bc', 'mths_since_recent_bc_dlq',
            'mths_since_recent_inq', 'mths_since_recent_revol_delinq',
            'pct_tl_nvr_dlq','sec_app_mths_since_last_major_derog']
fill_min = np.setdiff1d(df.columns.values, np.append(fill_empty, fill_max))

df[fill_empty] = df[fill_empty].fillna('none')
df[fill_max] = df[fill_max].fillna(df[fill_max].max())
df[fill_min] = df[fill_min].fillna(df[fill_min].min())

In [ ]:
#df['daysOpenLastYear']=df.groupby('Region').daysOpenLastYear.transform(lambda x: x.fillna(x.mean()))

In [ ]:
to_drop = ['hardship_start_date', 'hardship_end_date', 'payment_plan_start_date', 'hardship_reason', 'settlement_date', 'debt_settlement_flag_date', 'next_pymnt_d', 'last_pymnt_d', 'last_credit_pull_d' ]
df = df.drop(to_drop, axis=1, errors='ignore')

In [ ]:
nas=pd.DataFrame(df.isnull().sum().sort_values(ascending=False)/len(df),columns = ['percent'])
pos = nas['percent'] > 0
nas[pos]

In [ ]:
num_feat = df.select_dtypes('number').columns.values
df[num_feat].nunique().sort_values()

In [ ]:
df = df.drop(['id'], axis=1, errors='ignore')

In [ ]:
#from itertools import combinations
#from scipy.stats import pearsonr, chi2_contingency

In [ ]:
#num_feat = X.select_dtypes('number').columns.values
#comb_num_feat = np.array(list(combinations(num_feat, 2)))
#corr_num_feat = np.array([])
#for comb in comb_num_feat:
#    corr = pearsonr(X[comb[0]], X[comb[1]])[0]
#    corr_num_feat = np.append(corr_num_feat, corr)

In [ ]:
#high_corr_num = comb_num_feat[np.abs(corr_num_feat) >= 0.9]
#high_corr_num

In [ ]:
#X = X.drop(np.unique(high_corr_num[:, 0]), axis=1, errors='ignore')

In [ ]:
#cat_feat = X.select_dtypes('object').columns.values
#X[cat_feat].nunique().sort_values()

In [ ]:
df = df.drop(['url', 'emp_title'], axis=1, errors='ignore')

In [ ]:
#cat_feat = X.select_dtypes('object').columns.values
#comb_cat_feat = np.array(list(combinations(cat_feat, 2)))
#corr_cat_feat = np.array([])
#for comb in comb_cat_feat:
#    table = pd.pivot_table(X, values='loan_amnt', index=comb[0], columns=comb[1], aggfunc='count').fillna(0)
#    corr = np.sqrt(chi2_contingency(table)[0] / (table.values.sum() * (np.min(table.shape) - 1) ) )
#    corr_cat_feat = np.append(corr_cat_feat, corr)

In [ ]:
df.head()

In [ ]:
df['loan_status'].value_counts()

In [ ]:
df['good_bad'] = df['loan_status'].isin(['Current', 'Fully Paid', 'In Grace Period']).astype('int')

In [ ]:
df['good_bad'].value_counts()

In [ ]:
df.to_csv('Lending_Tree_Cleaned.csv')